# Latent Dirichlet Allocation LDA 

#### Wikifetcher
Raw Text von Wikipedia mittels Suchbegriffen
#### LDAbuilder
Ausführen der LDA mit der gegebenen Dokumentliste (Rohtext-Liste von Wikifetcher)

## Ausführung
### Konfiguration 
- Wir benötigen Zugriff auf Wikipedia für den Rohtext
- Natural Language Toolkit NLTK für die Tokenisierung und Stemming
- Stop_words, um nichtssagende Wörter zu entfernen
- Gensim für die Implementierung der Latent Dirichlet Allocation LDA

In [1]:
import wikipedia
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim import corpora, models

sentence_pat = re.compile(r'([A-Z][^\.!?]*[\.!?])', re.M)
tokenizer = RegexpTokenizer(r'\w+')

# Erzeuge englische stop words Liste
en_stop = get_stop_words('en')
# Erzeuge p_stemmer der Klasse PorterStemmer
p_stemmer = PorterStemmer()

doc_list = []
wikipedia.set_lang('en')

### Wikipedia Content
Mittels Suchbegriffen holen wir den Rohen Inhalt aus Wikipedia.
Danach wird der Inhalt in Sätze getrennt, welche zur Dokumentliste hinzugefügt werden.

In [2]:
def get_page(name):
    first_found = wikipedia.search(name)[0]
    try:
        return(wikipedia.page(first_found).content)
    except wikipedia.exceptions.DisambiguationError as e:
        return(wikipedia.page(e.options[0]).content)
    
search_terms = ['Stuttgart']
separator = '== References =='
for term in search_terms:
    full_content = get_page(term).split(separator, 1)[0]
    sentence_list = sentence_pat.findall(full_content)
    for sentence in sentence_list:
        doc_list.append(sentence)

    print(full_content[0:1500] + '...')

Stuttgart ( SHTUUT-gart; German: [ˈʃtʊtɡaʁt] ( listen); Swabian: Schduagert, pronounced [ˈʒ̊d̥ua̯ɡ̊ɛʕd̥]; names in other languages) is the capital and largest city of the German state of Baden-Württemberg.
Stuttgart is located on the Neckar river in a fertile valley locally known as the "Stuttgart Cauldron" an hour from the Swabian Jura and the Black Forest, and its urban area has a population of 609,219, making it the sixth largest city in Germany. 2.7 million people live in the city's administrative region and another 5.3 million people in its metropolitan area, making it the fourth largest metropolitan area in Germany.
The city and metropolitan area are consistently ranked among the top 20 European metropolitan areas by GDP; Mercer listed Stuttgart as 21st on its 2015 list of cities by quality of living, innovation agency 2thinknow ranked the city 24th globally out of 442 cities  and the Globalization and World Cities Research Network ranked the city as a Beta-status world city in t

### Vorverarbeitung
Der Text wird nun Tokenisiert, gestemt, nutzlose Wörter werden entfernt

In [3]:
num_topics = 4
num_words_per_topic = 15
texts = []

In [4]:
import pandas as pd

for doc in doc_list:
    raw = doc.lower()
    # Erzeuge tokens
    tokens = tokenizer.tokenize(raw)
    # Entferne unnütze Information
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # Stemme tokens - Entfernung von Duplikaten und Transformation zu Grundform
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    texts.append(stemmed_tokens)
output_preprocessed = pd.Series(texts)
print(output_preprocessed)

0      [stuttgart, shtuut, gart, german, ˈʃtʊtɡaʁt, l...
1      [stuttgart, locat, neckar, river, fertil, vall...
2                                              [germani]
3      [citi, metropolitan, area, consist, rank, amon...
4      [sinc, 6th, millennium, bc, stuttgart, area, i...
5      [roman, empir, conquer, area, 83, ad, built, m...
6      [stuttgart, s, root, truli, laid, 10th, centur...
7      [overshadow, nearbi, cannstatt, town, grew, st...
8      [fortun, stuttgart, turn, hous, württemberg, m...
9      [stuttgart, prosper, despit, setback, thirti, ...
10     [howev, 1952, citi, bounc, back, becam, major,...
11     [stuttgart, also, transport, junction, possess...
12              [stuttgart, unusu, scheme, german, citi]
13     [spread, across, varieti, hill, cover, vineyar...
14     [often, sourc, surpris, visitor, associ, citi,...
15          [citi, s, tourism, slogan, stuttgart, offer]
16     [current, plan, improv, transport, link, inter...
17                    [busi, de

### Dictionary und Vektoren
In diesem Abschnitt wird nun der Bag-of-words Korpus erstellt. Die Vektoren werden später für das LDA-Modell benötigt

In [5]:
# Erzeuge ein dictionary
dictionary = corpora.Dictionary(texts)
# Konvertiere dictionary in Bag-of-Words
# corpus ist eine Liste von Vektoren - Jeder Dokument-Vektor ist eine Serie von Tupeln
corpus = [dictionary.doc2bow(text) for text in texts]

output_vectors = pd.Series(corpus)
print(output_vectors)

0      [(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1...
1      [(0, 2), (6, 1), (17, 1), (18, 1), (22, 1), (2...
2                                              [(41, 1)]
3      [(0, 1), (18, 7), (35, 2), (42, 2), (43, 1), (...
4      [(0, 1), (23, 1), (26, 1), (35, 2), (69, 1), (...
5      [(35, 1), (39, 1), (73, 1), (82, 1), (83, 1), ...
6      [(0, 1), (96, 1), (97, 1), (98, 1), (99, 1), (...
7      [(92, 1), (109, 1), (110, 1), (111, 1), (112, ...
8      [(0, 1), (16, 1), (21, 1), (96, 1), (117, 1), ...
9      [(0, 1), (18, 1), (126, 1), (127, 1), (128, 1)...
10     [(18, 1), (94, 1), (138, 1), (139, 1), (140, 1...
11     [(0, 1), (17, 1), (40, 1), (41, 1), (149, 1), ...
12         [(0, 1), (3, 1), (18, 1), (164, 1), (165, 1)]
13     [(0, 1), (23, 1), (24, 1), (26, 1), (166, 1), ...
14     [(18, 1), (136, 1), (176, 1), (177, 1), (178, ...
15     [(0, 1), (18, 1), (97, 1), (146, 1), (183, 1),...
16     [(0, 1), (18, 1), (150, 1), (183, 1), (185, 1)...
17              [(199, 1), (206

### LDA-Modell
Schließlich kann das LDA-Modell angewandt werden. Die Übergabeparameter dafür sind die Liste der Vektoren, die Anzahl der Themen, das Dictionary, sowie die Aktualisierungsrate.
In der Trainingsphase sollte eine höhere Aktualisierungsrate >= 20 gewählt werden.

In [7]:
# Wende LDA-Modell an
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=20)
lda = ldamodel.print_topics(num_topics=num_topics, num_words=num_words_per_topic)
    
for topic in lda:
    for entry in topic:
        print(entry)

0
0.030*"stuttgart" + 0.025*"citi" + 0.008*"württemberg" + 0.007*"s" + 0.007*"district" + 0.005*"also" + 0.005*"state" + 0.005*"area" + 0.004*"archiv" + 0.004*"librari" + 0.004*"german" + 0.004*"year" + 0.004*"new" + 0.004*"today" + 0.003*"intern"
1
0.025*"stuttgart" + 0.011*"s" + 0.010*"citi" + 0.005*"germani" + 0.005*"german" + 0.004*"last" + 0.004*"state" + 0.004*"000" + 0.004*"one" + 0.004*"centuri" + 0.003*"war" + 0.003*"later" + 0.003*"also" + 0.003*"home" + 0.003*"duke"
2
0.036*"stuttgart" + 0.012*"citi" + 0.012*"württemberg" + 0.007*"s" + 0.006*"world" + 0.006*"univers" + 0.006*"baden" + 0.005*"german" + 0.005*"germani" + 0.005*"also" + 0.005*"area" + 0.004*"state" + 0.004*"cannstatt" + 0.004*"center" + 0.004*"found"
3
0.025*"stuttgart" + 0.013*"citi" + 0.009*"s" + 0.007*"us" + 0.006*"germani" + 0.006*"area" + 0.005*"local" + 0.005*"sinc" + 0.005*"museum" + 0.005*"armi" + 0.005*"year" + 0.005*"war" + 0.004*"württemberg" + 0.004*"german" + 0.004*"town"
